# Calculate the total number of active cases as of the dates mentioned in file Case Data.xlsx (Tab - Total Active Cases)


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [129]:
total_active_cases_df = pd.read_excel('Case Data.xlsx', sheet_name='Total Active Cases')

In [130]:
# Converting the 'Date as of' column to datetime
total_active_cases_df['Date as of'] = pd.to_datetime(total_active_cases_df['Date as of'])

In [131]:
# Extracting the data for the curve fitting
dates = total_active_cases_df['Date as of']
active_cases = total_active_cases_df['Active Cases']

In [132]:
total_active_cases_df.dropna(inplace=True)

In [133]:
# Converting dates to ordinal for curve fitting
x_data = dates.map(pd.Timestamp.toordinal).values
y_data = active_cases.values

In [134]:
y_data.shape, x_data.shape

((24,), (24,))

In [141]:
total_active_cases_df

,Date as of,Active Cases
0,2018-10-21,1.0
1,2018-11-20,7.0
2,2018-12-20,5.0
3,2019-01-19,6.0
4,2019-02-18,6.0
5,2019-03-20,7.0
6,2019-04-19,9.0
7,2019-05-19,5.0
8,2019-06-18,11.0
9,2019-07-18,7.0


In [142]:
total_active_cases_df.isnull().sum()

Date as of      0
Active Cases    0
dtype: int64

In [143]:
# Identifying inf and NaN values
print("Original x:", x_data)
print("Original y:", y_data)

# Removing or replacing the inf and NaN values
# removing the entries with inf or NaN values
mask = ~np.isnan(x_data) & ~np.isnan(y_data) & ~np.isinf(x_data) & ~np.isinf(y_data)
x_clean = x_data[mask]
y_clean = y_data[mask]

print("Cleaned x:", x_clean)
print("Cleaned y:", y_clean)

Original x: [736988 737018 737048 737078 737108 737138 737168 737198 737228 737258
 737288 737318 737348 737378 737408 737438 737468 737498 737528 737558
 737588 737618 737648 737678]
Original y: [ 1.  7.  5.  6.  6.  7.  9.  5. 11.  7.  7.  0.  0. 16. 16.  9.  9.  9.
 nan nan nan nan nan nan]
Cleaned x: [736988 737018 737048 737078 737108 737138 737168 737198 737228 737258
 737288 737318 737348 737378 737408 737438 737468 737498]
Cleaned y: [ 1.  7.  5.  6.  6.  7.  9.  5. 11.  7.  7.  0.  0. 16. 16.  9.  9.  9.]


In [183]:
def logistic(x_clean, a, b, c):
    return c / (1 + np.exp(-(x_clean - b) / a))

# Fitting the logistic function to the data
params, _ = curve_fit(logistic, x_clean, y_clean, maxfev=10000)

In [219]:
future_dates = pd.date_range(start=dates.max(), periods=6, freq='M').map(pd.Timestamp.toordinal).values
forecast = logistic(future_dates, *params)

In [225]:
# Converting future_dates back to datetime
future_dates = pd.to_datetime(future_dates, origin='unix', unit=None)

In [229]:
# Preparing the results
forecast_df = pd.DataFrame({'Date': future_dates, 'Forecasted Active Cases': forecast})
forecast_df

,Date,Forecasted Active Cases
0,1970-01-01 00:00:00.000737698,7.222222
1,1970-01-01 00:00:00.000737729,7.222222
2,1970-01-01 00:00:00.000737759,7.222222
3,1970-01-01 00:00:00.000737790,7.222222
4,1970-01-01 00:00:00.000737821,7.222222
5,1970-01-01 00:00:00.000737849,7.222222


In [232]:
forecast_df.to_csv('C:/Users/maniesh/Landmark/DLL Take Home Exercise/Forecasted_ActiveCases.csv', index=False)